In [ ]:
```
22.03.07
1. 보팅(xgboost + LGBM + RF) => 0.8278
2. pycaret(catboost제외) => 0.8415
3. 보팅 (extratree + LGBM + catboost) => 0.8411

22.03.08
1. pycaret(catboost포함) => 0.8464 
2. 보팅 (extratree + LGBM + catboost)- 파라미터 추가 => 0.847

22.03.10
1. cnn => 0.8916  
2. cnn(+dropout, adam) => 0.8788
3. cnn(dropout+0.03) => 0.8850

22.03.11
1. cnn(+dropout) => 0.8811
2. cnn(+patient=10) => 0.8918
3. cnn(+patient=50) => 0.8963 

22.03.12
1. cnn(Nadam) => 0.8961
2. cnn(adam) => 0.8944
3. cnn(filter) => 0.8925

22.03.13
1. cnn(차원) => 0.8961
2. CNN(flatten) => 0.8935

22.03.14
1. cnn(필터 수 + 콜백 + nadam) => 0.8869
2. cnn(dense층 여러개 + dropout) => 0.8927
3. cnn(dense=32, dropout=0.5) => 0.8931

22.03.15
1. cnn(adam학습률 + 배치 16) => 0.8850
2. cnn(레이어 수 추가, 필터갯수 변경, dense추가) => 0.8957
3. cnn(val_loss) => 0.8843

22.03.16
1. cnn(epoch=200) => 0.8959
2. cnn(RMSprop 학습률 0.001) => 0.8933

22.03.17
1. cnn(직사각 필터 21 21 32 32) => 0.90023 
2. cnn(직사각 필터 21 21 32 43) => 0.90021
3. cnn(직사각 필터 21 21 32 32) => 0.8942

22.03.18
1. cnn(콜백 pa5 0.8852) => 0.900663
2. cnn(차원 41 31 32 22 0.8864) => 0.90730
3. cnn(팩터 0.2 24 24 34 34 0.8972) => 0.90751 (best)
```


# 보팅 (xgboost + lgbm + rf)

In [88]:
import pandas as pd
import os
import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


3.4.3


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(train_X,train_y)

In [ ]:
pred = VC.predict(test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = pred

submission

submission.to_csv('submission.csv',index=False)

# pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
setup_clf = setup(data = train, target = 'target', fold_shuffle=True, use_gpu=True, silent=True)

In [ ]:
top10 = compare_models(sort='Accuracy', n_select=10, fold = 5)

In [ ]:
tuned_top10 = [tune_model(i, fold = 5) for i in top10]

In [ ]:
blender_top10 = blend_models(estimator_list=tuned_top10)

In [ ]:
final_model = finalize_model(blender_top10)
prediction = predict_model(final_model, data= test)

In [ ]:
prediction

In [ ]:
prediction['Label']

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = prediction['Label']

submission

submission.to_csv('submission.csv',index=False)

In [ ]:
# extratree => 0.87
# xgboost => 0.87

# 보팅 (extratree + LGBM + catboost)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import catboost as ctb

In [ ]:
LGBM = LGBMClassifier(max_bin=300, num_leaves=24)
xtree = ExtraTreesClassifier(max_depth=6)
ctb = ctb.CatBoostClassifier(silent=True, depth=6, l2_leaf_reg=7, learning_rate=0.1)

VC = VotingClassifier(estimators=[('xt',xtree),('ctb',ctb),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(train_X, train_y)

In [ ]:
pred = VC.predict(test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 정확도 계산 - test셋을 분리 후 정확도 보기

## 보팅으로 정확도 보기

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8373

## ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
xtree = ExtraTreesClassifier()
xtree.fit(X_train, y_train)

In [ ]:
pred = xtree.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8030

## 보팅으로 정확도 보기2 (RF -> extratree)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
xtree = ExtraTreesClassifier()

VC = VotingClassifier(estimators=[('xt',xtree),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8415

## 랜덤포레스트로 정확도 보기

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.7709

## xgboost로 정확도 보기

In [ ]:
import xgboost as xgb 

In [ ]:
dtrain = xgb.DMatrix(data=X_train , label=y_train)
dtest = xgb.DMatrix(data=X_test , label=y_test)

In [ ]:
params = { 'max_depth':3,
           'eta': 0.1,   # 학습률
           'objective':'multi:softmax',
           'eval_metric':'merror',
           'early_stoppings':10
        }

# 학습 횟수
num_rounds = 100

In [ ]:
# 데이터를 학습용은 ‘train’ , 검증용은 ‘eval’로 표시
wlist = [(dtrain,'train'),(dtest,'eval') ]

In [ ]:
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)  #xgb.fit이 아닌 xgb.train
xgb_model

## Catboost

In [ ]:
import catboost as ctb

In [ ]:
model_CBC = ctb.CatBoostClassifier()

In [ ]:
model_CBC.fit(X_train, y_train)

In [ ]:
pred = model_CBC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

#예측 정확도: 0.8458

## 보팅으로 정확도 보기2 (xgboost => catboost)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import catboost as ctb

In [ ]:
LGBM = LGBMClassifier(max_bin=100, num_leaves=15, max_depth=6)
xtree = ExtraTreesClassifier(max_depth=6)
ctb = ctb.CatBoostClassifier(silent=True, depth=6, l2_leaf_reg=7, learning_rate=0.1)


VC = VotingClassifier(estimators=[('xt',xtree),('ctb',ctb),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8672

# 딥러닝 기반

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping
import tensorflow as tf

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [11]:
model = Sequential()
model.add(Dense(40, input_dim=32, activation = 'relu'))
model.add(Dense(20, activation='relu'))
#model.add(Dense(50, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=30, batch_size=3)

In [ ]:
# 예측 값과 실제 값의 비교
Y_prediction = model.predict(test)
# flatten : 데이터 배열이 몇 차원이든 모두 1차원으로 바꿔 읽기 쉽게 해 주는 함수

In [ ]:
value_list=[]
for i in range(len(Y_prediction)):
    maxindex = np.argmax(Y_prediction[i])
    value_list.append(maxindex)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, value_list)))


# 예측 정확도: 0.6146

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = value_list

submission

submission.to_csv('submission.csv',index=False)

In [ ]:
submission.target2.value_counts()

In [ ]:
submission.target.value_counts()

## 파라미터 튜닝

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [ ]:
model_CAT = ctb.CatBoostClassifier(silent = True)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_cv_CAT = GridSearchCV(model_CAT, param_grid=grid, cv=3 , n_jobs=-1)
grid_cv_CAT.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_CAT.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_CAT.best_score_))

In [ ]:
model_XGB = XGBClassifier(eval_metric='logloss', silent = True)

param_grid={'booster' :['gbtree'],
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'n_estimators':[50],
                 'objective':['multi:softmax'],
                 'random_state':[2]}

grid_cv_XGB=GridSearchCV(model_XGB, param_grid=param_grid, cv=5 , n_jobs=-1)
grid_cv_XGB.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_XGB.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_XGB.best_score_))

# CNN - 기세현님 코드 참고

In [90]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [91]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [92]:
X = train.iloc[:, 1:-1]
target = test.iloc[:, 1:]

In [93]:
X = np.array(X).reshape(-1, 8, 4, 1)
target = np.array(target).reshape(-1, 8, 4, 1)

In [94]:
ohe = OneHotEncoder(sparse = False)
y = ohe.fit_transform(train[['target']])

In [95]:
skf = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

In [112]:
es = EarlyStopping(monitor = 'val_acc', patience = 50, mode = 'max', verbose = 0)
rlrp = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, mode='max')

In [116]:
cnn_acc = []
cnn_pred = np.zeros((target.shape[0], 4))
for i, (tr_idx, val_idx) in enumerate(skf.split(X, train.target)) :
    print(f'{i + 1} Fold Training.....')
    tr_x, tr_y = X[tr_idx], y[tr_idx]
    val_x, val_y = X[val_idx], y[val_idx]
    
    ### CNN 모델
    cnn = Sequential()
    cnn.add(Conv2D(64, (2, 4), padding = "same", activation = 'elu', input_shape = (8, 4, 1)))
    cnn.add(BatchNormalization())
    
    cnn.add(Conv2D(32, (2, 4), padding = "same", activation = 'elu'))
    cnn.add(BatchNormalization())
    
    cnn.add(Conv2D(16, (3, 4), padding = "same", activation = 'elu'))
    cnn.add(BatchNormalization())
    
    cnn.add(Conv2D(32, (3, 4), padding = "same", activation = 'elu'))
    
    cnn.add(BatchNormalization())
    cnn.add(GlobalAveragePooling2D())
    
    cnn.add(Dense(16, activation = 'relu'))
    cnn.add(Dense(4, activation = 'softmax'))

    ### ModelCheckPoint Fold마다 갱신
    mc = ModelCheckpoint(f'model_{i + 1}.h5', save_best_only = True, monitor = 'val_acc', mode = 'max', verbose = 0)
   
    ### 모델 compile
    cnn.compile(optimizer = RMSprop(lr=0.04), loss = 'categorical_crossentropy', metrics = ['acc'])

    cnn.fit(tr_x, tr_y, validation_data = (val_x, val_y), epochs = 100, batch_size = 32, callbacks = [es, mc, rlrp], verbose = 0)

    ### 최고 성능 기록 모델 Load
    best = load_model(f'model_{i + 1}.h5')
    ### validation predict
    val_pred = best.predict(val_x)
    ### 확률값 중 최대값을 클래스로 매칭
    val_cls = np.argmax(val_pred, axis = 1)
    ### Fold별 정확도 산출
    fold_cnn_acc = accuracy_score(np.argmax(val_y, axis = 1), val_cls)
    cnn_acc.append(fold_cnn_acc)
    print(f'{i + 1} Fold ACC of CNN = {fold_cnn_acc}\n')

    ### Fold별 test 데이터에 대한 예측값 생성 및 앙상블
    fold_pred = best.predict(target) / skf.n_splits
    cnn_pred += fold_pred

1 Fold Training.....
1 Fold ACC of CNN = 0.8717948717948718

2 Fold Training.....
2 Fold ACC of CNN = 0.9145299145299145

3 Fold Training.....
3 Fold ACC of CNN = 0.8974358974358975

4 Fold Training.....
4 Fold ACC of CNN = 0.8760683760683761

5 Fold Training.....
5 Fold ACC of CNN = 0.8974358974358975

6 Fold Training.....
6 Fold ACC of CNN = 0.8841201716738197

7 Fold Training.....
7 Fold ACC of CNN = 0.8755364806866953

8 Fold Training.....
8 Fold ACC of CNN = 0.9098712446351931

9 Fold Training.....
9 Fold ACC of CNN = 0.9012875536480687

10 Fold Training.....
10 Fold ACC of CNN = 0.8454935622317596



In [117]:
np.mean(cnn_acc)

# 0.8749
# 0.8757
# 0.8689

0.8873573970140495

In [105]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = np.argmax(cnn_pred, axis = 1)

submission

submission.to_csv('submission.csv',index=False)

In [89]:
from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11849014223694398871]